# Type Hinting & MyPy 학습
- `Typing`을 통한 코드 문법 실습
- `MyPy`를 통한 오류 검수
- 데이터 엔지니어링 파이프라인 설계 학습

| **단계**       | **주요 내용**                           | **커리 큘럼**    |
| ------------ | ----------------------------------- | ------------ |
| **1. 기초 설계** | Pydantic 모델링, Generic을 이용한 추상화      | **진행 예정** 🔄     |
| **2. 아키텍처**  | Protocol을 이용한 인터페이스 설계, ETL 구조      | **진행 예정** 🔄     |
| **3. 안정성**   | Try-Except를 활용한 DLQ 전략, 에러 분류       | **진행 예정** 🔄  |
| **4. 최적화**   | **비동기(Async) 처리, gather, 세마포어(제한)** | **진행 예정** 🔜 |
| **5. 실전 응용** | 실제 API 호출 연동 및 대용량 데이터 시뮬레이션        | **최종 목표** 🏁 |

## 1. 변수와 함수의 기초 타입 지정

In [1]:
# 변수 타입 지정
user_id: int = 12345
user_name: str = "Gemini"

# 함수 타입 지정
def greet(name: str) -> str:
    return f"Hello, {name}!"

## 2. 복잡한 구조를 위한 `typing` 모듈

In [2]:
from typing import List, Dict, Optional, Union

# 정수들로 이루어진 리스트
scores: List[int] = [90, 85, 100]

# 키는 문자열, 값은 정수인 딕셔너리
user_ages: Dict[str, int] = {"Alice": 25, "Bob": 30}

# 문자열이거나 None일 수 있는 경우
middle_name: Optional[str] = None

In [3]:
# 2개의 정수를 받은 후 합산한 값을 반환하는 함수
def add_numbers(a: int, b: int) -> int:
    return a + b

# 결과 확인
result = add_numbers(10, 20)
print(result)

# str 타입을 인수로 받을 경우는?
result = add_numbers("10", "이십")
print(result)

30
10이십


In [4]:
# 정수 리스트를 받은 후 (합산 / 리스트의 개수)를 반환 
def calculate_average(numbers: list[int]) -> int:
    return sum(numbers) / len(numbers)

# 결과 확인
nums = [10, 20, 30]
print(calculate_average(nums))

20.0


In [5]:
# 1. 정수 연산만 허용하도록 정의
def add_numbers(a: int, b: int) -> int:
    return a + b

# 2 & 3. float 반환 및 빈 리스트 처리(Optional)
def calculate_average(numbers: List[int]) -> Optional[float]:
    if not numbers:  # 리스트가 비어있을 경우
        return None
    return sum(numbers) / len(numbers)

# 정상 호출
print(add_numbers(10, 20))           # 결과: 30
print(calculate_average([10, 20, 30]))  # 결과: 15.0

30
20.0


In [6]:
# 정수(int)와 실수(float)을 모두 담을 수 있는 리스트
def process_nbs(data: List[Union[int, float]]) -> None:
    for item in data:
        print(f"처리 중: {item}")
        
# 1. 정수와 실수가 섞인 리스트를 넣을 시 / "40"의 리스트 str 타입 인수 값은 MyPy에서 오류로 검수 됨.
process_nbs([10, 20.5, 30, "40"])

처리 중: 10
처리 중: 20.5
처리 중: 30
처리 중: 40


## 3. 현업의 표준 Pydantic과 MyPy
- 단순한 타입 체크를 넘어서 들어오는 데이터의 형식을 강제하고 검증(Validation)하는 방법 알아보기
- Pydantic + MyPy
    + Pydantic: `"123"`이라는 문자열이 들어와도 int 타입 선언을 보고 `123`으로 자동 형변환
    + IDE 지원: MyPy 덕분에 코드를 잘 때 `user1.` 까지만 타이핑해도 `id`, `name` 같은 필드명이 자동 완성
    + 복잡한 중첩 구조: 리스트 안에 딕셔너리가 있고, 그 안에 또 리스트가 있는 복잡한 JSON 데이터도 명확하게 타입을 정의할 수 있다.

In [7]:
from pydantic import BaseModel
from typing import List, Dict, Optional, Union

# 데이터 구조(스키마) 정의
class User(BaseModel):
    id: int
    name: str
    email: Optional[str] = None
    friends: List[int] = []

# 1. 정상적인 데이터 생성
user1 = User(id=1, name="Alice", email="Alice@example.com")
print(user1.id) # MyPy는 user1.id가 int임을 알고 있습니다.

# 2. 잘못된 데이터가 들어올 경우 (실행 시 에러 발생)
# user2 = User(id="wrong", name="Bob") # id에 숫자가 아닌 값이 오면 Pydantic이 막아줌

1


In [8]:
class Order(BaseModel):
    order_id: int
    price: float
    tags: List[str] = []

# 테스트 데이터
my_order = Order(order_id = 101, price = 15.5, tags=["eletronics", "gift"])
print(my_order)

order_id=101 price=15.5 tags=['eletronics', 'gift']


In [9]:
# 숫자가 아닌 문자열을 기입하면? >>> 자동 형 변환
my_order = Order(order_id = "101", price = 15.5, tags=["eletronics", "gift"])
print(my_order)

order_id=101 price=15.5 tags=['eletronics', 'gift']


In [10]:
# 필수 항목인 order_id를 누락한다면? >>> `ValidationError` 에러 발생하여 프로그램 중단
# my_order = Order(price = 15.5, tags=["eletronics", "gift"])
# print(my_order)
"""
---------------------------------------------------------------------------
ValidationError                           Traceback (most recent call last)
Cell In[10], line 2
      1 # 필수 항목인 order_id를 누락한다면? >>> `ValidationError` 에러 발생하여 프로그램 중단
----> 2 my_order = Order(price = 15.5, tags=["eletronics", "gift"])
      3 print(my_order)

File ~/venv/lib/python3.11/site-packages/pydantic/main.py:250, in BaseModel.__init__(self, **data)
    248 # `__tracebackhide__` tells pytest and some other tools to omit this function from tracebacks
    249 __tracebackhide__ = True
--> 250 validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
    251 if self is not validated_self:
    252     warnings.warn(
    253         'A custom validator is returning a value other than `self`.\n'
    254         "Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.\n"
    255         'See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.',
    256         stacklevel=2,
    257     )

ValidationError: 1 validation error for Order
order_id
  Field required [type=missing, input_value={'price': 15.5, 'tags': ['eletronics', 'gift']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
"""

'\n---------------------------------------------------------------------------\nValidationError                           Traceback (most recent call last)\nCell In[10], line 2\n      1 # 필수 항목인 order_id를 누락한다면? >>> `ValidationError` 에러 발생하여 프로그램 중단\n----> 2 my_order = Order(price = 15.5, tags=["eletronics", "gift"])\n      3 print(my_order)\n\nFile ~/venv/lib/python3.11/site-packages/pydantic/main.py:250, in BaseModel.__init__(self, **data)\n    248 # `__tracebackhide__` tells pytest and some other tools to omit this function from tracebacks\n    249 __tracebackhide__ = True\n--> 250 validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)\n    251 if self is not validated_self:\n    252     warnings.warn(\n    253         \'A custom validator is returning a value other than `self`.\n\'\n    254         "Returning anything other than `self` from a top level model validator isn\'t supported when validating via `__init__`.\n"\n    255         \'See the `mod

## 4. MyPy 설정
- MyPy 설정이란 프로젝트 전체의 '규칙'을 정하여 팀내 협업을 상승시키는 방법입니다.
- `mypy.ini`나 `project.toml` 파일을 만들어 MyPy 설정을 할 수 있습니다.

| **설정 키 🔑**                  | **역할 🛠️**            | **중요성 및 효과 💡**                                                                  |
| ---------------------------- | --------------------- | -------------------------------------------------------------------------------- |
| **`disallow_untyped_defs`**  | 타입 힌트가 없는 함수 정의 금지    | 모든 함수의 입력과 출력을 명확히 기록하도록 강제하여 코드의 가독성과 안정성을 높입니다.                                |
| **`ignore_missing_imports`** | 타입 정보가 없는 라이브러리 무시    | `pandas`나 `numpy`처럼 타입 힌트가 내장되어 있지 않은 외부 라이브러리에서 발생하는 불필요한 에러를 방지합니다.            |
| **`no_implicit_optional`**   | 암시적인 `Optional` 사용 금지 | `name: str = None` 대신 `Optional[str]`를 명시하게 하여, 값이 비어 있을 가능성을 개발자가 확실히 인지하게 합니다. |

## 5. 실전 데이터 처리(JSON LIST 처리)
 - 외부 API를 통하여 여러 명의 유저 정보가 담긴 JSON 리스트를 받았습니다.
 - 이 데이터를 파싱하여 유효한 유저 객체 리스트로 변환해야 합니다.

In [11]:
from pydantic import BaseModel, ValidationError
from typing import List, Optional

# 1. 유저 모델 정의
class User(BaseModel):
    id: int
    name: str
    age: Optional[int] = None

# 2. 유저 리스트 처리 함수
# 입력 값 타입 = 'raw_data'라는 리스트, 결과는 'User' 객체들의 리스트
def process_user_batch(raw_data: List[dict]) -> List[User]:
    valid_users: List[User] = []

    for item in raw_data:
        try:
            # Pydantic 모델로 변환 (자동 검증)
            user = User(**item)
            valid_users.append(user)
        except ValidationError as e:
            print(f"잘못된 데이터 스킵: {item} | 이유: {e}")
    
    return valid_users

# 3. 테스트 실행
data_from_api = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": "2", "name": "Bob"}, # id의 str 타입은 자동으로 int 형 변환됨. age는 Optional None 허용
    {"name": "Unknown", "age": 30} # id가 없어서 실패할 데이터
]

processed = process_user_batch(data_from_api)
print(f"성공적으로 처리된 유저 수: {len(processed)}")

잘못된 데이터 스킵: {'name': 'Unknown', 'age': 30} | 이유: 1 validation error for User
id
  Field required [type=missing, input_value={'name': 'Unknown', 'age': 30}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
성공적으로 처리된 유저 수: 2


### 데이터 파이프라인 종합 실습
웹사이트 로그 데이터를 받아서 분석용 데이터로 변환하는 작업 수행

조건: 
1. 로그 모델(`LogRecord`): `timestamp(str)`, `event_type(str)`, `user_id(Optional[int])` 필드를 가집니다. 
2. 클렌징 함수: 로그 데이터 리스트를 받아서, `user_id`가 있는 데이터만 골라내어 `LogRecord` 객체 리스트로 반환합니다.

In [12]:
from pydantic import BaseModel, ValidationError
from typing import List, Optional

# 1. 모델 정의
class LogRecord(BaseModel):
    timestamp: str
    event_type: str
    user_id: Optional[int] = None

# 2. 파이프라인 함수
def clean_logs(raws_logs: List[dict]) -> List[LogRecord]:
    clean_data: List[LogRecord] = []

    for log in raw_logs:
        try:
            record = LogRecord(**log)
            # user_id가 존재하는 경우만 리스트에 추가
            if record.user_id is not None:
                clean_data.append(record)
        except ValidationError:
            continue
    
    return clean_data

# 테스트 데이터
raw_logs = [
    {"timestamp": "2025-12-25 17:40", "event_type": "click", "user_id": 123},
    {"timestamp": "2025-12-25 17:41", "event_type": "view"}, # user_id 값 Null
    {"timestamp": "2025-12-25 17:42", "event_type": "login", "user_id": "456"} # user_id int로 자동 형 변환
]

# 결과 확인
final_logs = clean_logs(raw_logs)
for item in final_logs:
    print(f"클렌징 된 로그: {(item)}")
print(f"클렌징 된 로그 수: {len(final_logs)}")

클렌징 된 로그: timestamp='2025-12-25 17:40' event_type='click' user_id=123
클렌징 된 로그: timestamp='2025-12-25 17:42' event_type='login' user_id=456
클렌징 된 로그 수: 2


## 6. 데이터 엔지니어링 타입 관리 아키텍쳐
- 실무에서는 데이터가 파이프라인을 통과할 때 마다 타입의 성격이 조금씩 변합니다.
- 그리고 다음과 같은 패턴을 사용합니다.
    + Interface(추상화) 활용: 데이터 소스가 S3이든, MySQL이든 관계없이 동일한 방식으로 데이터를 읽어올 수 있게 설계합니다.
    + Generic 활용: 어떤 종류의 데이터 모델이 들어오더라도 동일한 클렌징 로직을 적용할 수 있는 '범용 처리기'를 만듭니다.

| **단계**                     | **타입 관리 전략 🛠️**              | **역할 및 도구 💡**                                       |
| -------------------------- | ----------------------------- | ---------------------------------------------------- |
| **1. Extraction (추출)**     | **Raw Type (Any/Dict)**       | 원천 데이터(JSON, CSV 등)를 그대로 가져옵니다. 이때는 타입이 불분명할 수 있습니다. |
| **2. Validation (검증)**     | **Strict Schema (Pydantic)**  | 외부 데이터를 우리가 정의한 모델(Pydantic)에 넣어 타입을 강제하고 정제합니다.     |
| **3. Transformation (변환)** | **Domain Models (Type Hint)** | 정제된 데이터를 바탕으로 비즈니스 로직을 처리합니다. 이때 제네릭과 추상화가 빛을 발합니다.  |
| **4. Loading (적재)**        | **Database Schema**           | 최종 타입을 DB 스키마(SQL 등)와 일치시켜 저장합니다.                    |

In [13]:
# Generic과 추상화(Abstraction): 유연한 코드 짜기
# Generic 학습

from pydantic import BaseModel
from typing import List, TypeVar, Optional

# 1. 제네릭 타입 변수 T를 정의. (BaseModel을 상속받은 클래스만 허용)
T = TypeVar("T", bound=BaseModel)

# 2. 범용 함수 정의
def filter_not_none(data_list: List[T], field_name: str) -> List[T]:
    result: List[T] = []

    for item in data_list:
        # getattr를 사용하면 문자열로 된 필드 이름으로 값에 접근할 수 있다.
        value = getattr(item, field_name)
        if value is not None:
            result.append(item)
    return result

# 3. 테스트 데이터
raw_logs = [
    {"timestamp": "2025-12-25 17:40", "event_type": "click", "user_id": 123},
    {"timestamp": "2025-12-25 17:41", "event_type": "view"}, # user_id 값 Null
    {"timestamp": "2025-12-25 17:42", "event_type": "login", "user_id": "456"} # user_id int로 자동 형 변환
]

# 4. Pydantic 모델 리스트로 변환 (검증)
log_objects = [LogRecord(**log) for log in raw_logs]

# 5. 제네릭 함수 호출
final_logs = filter_not_none(data_list = log_objects, field_name = "user_id")

# 6. 결과 출력
for item in final_logs:
    print(f"클렌징 된 로그: {item}")
print(f"클렌징 된 로그 수: {len(final_logs)}")

클렌징 된 로그: timestamp='2025-12-25 17:40' event_type='click' user_id=123
클렌징 된 로그: timestamp='2025-12-25 17:42' event_type='login' user_id=456
클렌징 된 로그 수: 2


### 추상화 아키텍쳐: 인터페이스와 구현의 분리

| **구성 요소**                  | **역할**                          | **비유 💡**                    |
| -------------------------- | ------------------------------- | ---------------------------- |
| **Protocol (Interface)**   | 동작의 규칙 정의 (`fetch_data`가 있어야 함) | 가전제품의 **220V 플러그 규격**        |
| **Implementation (Class)** | 실제 동작 구현 (S3에서 읽기, DB에서 읽기)     | 규격에 맞게 만들어진 **냉장고, TV, 세탁기** |
| **Client (Function)**      | 규격을 사용하는 로직 (`run_pipeline`)    | 플러그를 꽂아서 사용하는 **벽면 콘센트**     |

데이터 소스가 `Google Drive`로 바뀌더라도 `run_pipeline`는 수정할 필요 없다.

`GoogleDriveReader`만 넣으면 됨.

In [14]:
# 추상화(Abstraction) 학습
from typing import Protocol, List

class DataReader(Protocol):
    def fetch_data(self) -> List[dict]:        
        ... # ...(Ellipsis) = 구조만 정의
        ### 데이터를 읽어서 딕셔너리 리스트로 반환하는 규격 ###

class S3Reader:
    def fetch_data(self) -> List[dict]:
        print("S3에서 데이터를 가져오는 중...")
        return [{"timestamp": "2025-12-25", "event_type": "Click", "user_id": 1}]

class LocalFileReader:
    def fetch_data(self) -> List[dict]:
        print("로컬 파일에서 데이터를 읽는 중...")
        return [{"id": 1, "name": "S3_Alice", "age": 27}]

class GoogleReader:
    def fetch_data(self) -> List[dict]:
        print("구글 드라이브에서 데이터 읽는 중...")
        return [{"id": 1, "name": "Seoul Milk", "price": 3000}, 
                {"id": 2, "name": "Got Milk", "price": 2500},
                {"id": 3, "name": "Emart Milk", "price": 3500}]

def run_pipeline(reader: DataReader):
    data = reader.fetch_data()
    print(f"총{len(data)}개의 데이터를 처리했습니다.")

In [15]:
from pydantic import BaseModel
from typing import List, TypeVar, Type, Protocol

# 제네릭 타입 설정
T = TypeVar("T", bound=BaseModel)

# 범용 파이프라인 함수
def universal_pipeline(reader: DataReader, model_class: Type[T]) -> List[T]:
    # 1. reader를 통해 데이터를 가져옵니다.
    raw_data = reader.fetch_data()

    # 2. 가져온 raw_data(dict 리스트)를 모델 객체 리스트로 바꿉니다.
    return [model_class(**item) for item in raw_data]

In [16]:
# S3에서 로그 데이터를 가져와서 LogRecord 객체 리스트로 만들고 싶을 때
s3_reader = S3Reader()
logs = universal_pipeline(reader=s3_reader, model_class=LogRecord)

# 유저 데이터를 처리하고 싶다면?
user_reader = LocalFileReader() # 다른 Reader도 가능
users = universal_pipeline(reader=user_reader, model_class=User) # User 모델만 전달

S3에서 데이터를 가져오는 중...
로컬 파일에서 데이터를 읽는 중...


In [17]:
# 1. Product 모델 정의 (Pydantic)
class Product(BaseModel):
    id: int
    name: str
    price: int

# 2. 파이프라인 실행
product_reader = GoogleReader() # 상품 데이터가 S3에 있다고 가정
products = universal_pipeline(reader=product_reader, model_class=Product)

# 3. 결과 확인
for p in products:
    print(f"상품명: {p.name}, 가격: {p.price}")

구글 드라이브에서 데이터 읽는 중...
상품명: Seoul Milk, 가격: 3000
상품명: Got Milk, 가격: 2500
상품명: Emart Milk, 가격: 3500


## 7. 저장(Write) 과정
 - `DataWriter` 프로토콜

```python
# 3가지 부품을 조립해서 실행
full_pipeline(
    reader=EX_Reader,        # 📥 추출(Extract) 담당
    model_class=EX_Product,  # ✨ 변환(Transform) & 검증 담당
    writer=EX_Writer         # 📤 저장(Load) 담당
)


# 예외 상황(Error Handing) 추상화
    ## 1. Skip 전략: 에러가 난 데이터만 버리고 나머지는 계속 진행한다.
    ## 2. Stop 전략: 하나라도 에러가 나면 즉시 중단하고 관리자에게 알린다.
    ## 3. DLQ(Dead Letter Queue) 전략: 에러 난 데이터만 따로 모아서 나중에 다시 확인한다.
```

In [18]:
class DataWriter(Protocol):
    def save(self, data: List[BaseModel]) -> None:
        ### 데이터를 특정 목적지에 저장하는 규격 ###
        ...

# 구현 예시
class ConsoleWriter:
    def save(self, data: List[BaseModel]) -> None:
        print(f"--- [저장 완료] 총{len(data)}개의 아이템을 저장함 ---")

# 최종 파이프라인 함수 (저장 단계)
def full_pipeline(reader: DataReader, model_class: Type[T], writer: DataWriter) -> List[T]:
    # 1. 추출 & 변환
    items = universal_pipeline(reader, model_class)

    # 2. 저장
    writer.save(items)

## 8. DLQ(Dead Letter Queue) 전략
- `try-except` 구문을 활용하여 에러 상황 핸들링
- `universal_pipeline` 구조를 확장해서, 성공한 데이터와 실패한 데이터를 분리하는 반환 구조 만들기

| **발생 단계**          | **주요 에러 타입**                            | **원인 예시**                       |
| ------------------ | --------------------------------------- | ------------------------------- |
| **추출 (Extract)**   | `ConnectionError` / `FileNotFoundError` | 네트워크 끊김, S3 권한 문제, 파일 경로 오류     |
| **변환 (Transform)** | **`ValidationError`**                   | 데이터 타입 불일치, 필수 필드 누락 (Pydantic) |
| **저장 (Load)**      | `OperationalError` / `TimeoutError`     | DB 서버 다운, 저장 공간 부족, 쓰기 시간 초과    |
| **기타**             | `KeyError` / `TypeError`                | 딕셔너리에 없는 키 접근, 잘못된 연산 등         |

In [19]:
from typing import Tuple, List
from pydantic import ValidationError

# T는 이전에 정의한 TypeVar("T", bound=BaseModel)입니다.

def pipeline_with_dlq(
    reader: DataReader,
    model_class: Type[T]
) -> Tuple[List[T], List[dict]]: # (성공 리스트, 실패 리스트) 반환

    success_data: List[T] = []
    error_data: List[ditc] = []
    raw_data = reader.fetch_data()

    for item in raw_data:
        try:
            # 1. 검증 및 변환 시도
            instance = model_class(**item)
            success_data.append(instance)
        except ValidationError as e:
            # 2. 에러 발생 시 '실패 리스트'에 저장
            print(f"데이터 변환 실패! 이유: {e}")
            error_data.append(item)
            
    return succes_Data, error_Data

## 9. Pydantic Field로 값의 범위 제한하기
- `Field`를 사용하면 단순한 타입 체크를 넘어, 값의 크기나 문자열의 길이 등을 검증할 수 있습니다.

In [20]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    id: int
    name: str = Field(min_length=2) # min_length=2(최소 2글자 이상)
    price: int = Field(gt=0) # gt=0: Greater Than 0 (0보다 커야 함)

## 10. 비동기(Async) 파이프라인
- 성능을 극적으로 높이는 비동기(Async) 개념
- 실무에서는 API를 호출하거나 DB에 저장할 때 기다리는 시간(`I/O Wait`)이 많이 발생합니다.
- 이때 **비동기**를 사용하면 한 작업이 끝날 때까지 기다리지 않고 다른 작업을 미리 처리할 수 있습니다.

In [21]:
import asyncio #파이썬 표준 라이브러리로 설치할 필요 없음.

async def mock_api_call(i):
    print(f"[{i}] 요청 시작!")
    await asyncio.sleep(1) # 1초 동안 응답을 기다린다고 가정 (비동기 대기)
    return f"[{i}] 응답 완료"

# 주피터 환경에서는 'await'를 셀에서 바로 사용할 수 있습니다.
# 10개의 요청을 동시에 실행합니다.
results = await asyncio.gather(*(mock_api_call(i) for i in range(10)))
print(results)

[0] 요청 시작!
[1] 요청 시작!
[2] 요청 시작!
[3] 요청 시작!
[4] 요청 시작!
[5] 요청 시작!
[6] 요청 시작!
[7] 요청 시작!
[8] 요청 시작!
[9] 요청 시작!
['[0] 응답 완료', '[1] 응답 완료', '[2] 응답 완료', '[3] 응답 완료', '[4] 응답 완료', '[5] 응답 완료', '[6] 응답 완료', '[7] 응답 완료', '[8] 응답 완료', '[9] 응답 완료']


In [22]:
# 에러 핸들링: try-except 위치
# DLQ(Dead Letter Queue) 전략을 비동기 환경에 적용할 때 아래의 두 가지 선택지가 있다.
    ## 일괄 처리 후 검증: 모든 데이터를 다 가져온(await) 다음에 한꺼번에 루프를 돌며 검증한다.
    ## 가져오면서 즉시 검증: 개별 데이터를 가져오는 즉시 비동기 루틴 안에서 검증하고 성공/실패를 나눈다.(실무 선호 방식/각 요청 독립적 실패 가능성 존재)
from typing import Protocol, List
class AsyncDataReader(Protocol):
    # 데이터를 가져오는 과정이 비동기(async)임을 명시합니다.
    async def fetch_data(self) -> List[dict]:
        ...

async def async_universal_pipeline(
    reader: AsyncDataReader, model_class: Type[T]) -> List[T]:

    # 비동기로 데이터를 가져올 때까지 '기다려(await)' 줍니다.
    raw_data = await reader.fetch_data()

    return [model_class(**item) for item in raw_data]

In [23]:
async def process_item(raw_dict: dict, model_class: Type[T]):
    try:
        # Pydantic 모델로 변환 시도
        return model_class(**raw_dict)
    except ValidationERROR:
        # 검증 실패 시 DLQ로 보낼 데이터 반환(None이나 특수 객체)
        return "ERROR_DATA"

async def async_pipeline_with_dlq(reader, model_class):
    raw_data = await reader.fetch_data() # [{...}, {...}, ...]

    # 모든 데이터를 비동기적으로 동시에 '검증' 루틴에 태웁니다.
    tasks = [process_item(item, model_class) for item in raw_data]
    results = await asyncio.gather(*tasks)

    # 결과 분류
    success = [r for r in results if r != "ERROR_DATA"]
    errors = [item for r, item in zip(results, raw_data) if r == "ERROR_DATA"]

    return success, errors

In [24]:
async def async_run_pipeline():
    try:
        # 1. 네트워크 에러 발생 시 전체가 멈춰야 합니다.
        success, errors = await async_pipeline_with_dlq(s3_reader, Product)
        print(f"성공: {len(success)}건, 실패(DLQ): {len(errors)}건")

    except Exception as e:
        # 2. 시스템 에러(네트워크, 권한 등)는 여기서 통합 관리합니다.
        print(f"시스템 오류로 파이프라인 중단: {e}")

# 실행
await async_run_pipeline()

S3에서 데이터를 가져오는 중...
시스템 오류로 파이프라인 중단: object list can't be used in 'await' expression


| **에러 상황**                      | **발생 위치**             | **결과**                                           |
| ------------------------------ | --------------------- | ------------------------------------------------ |
| **데이터 형식 오류** (예: `price`가 음수) | `process_item` (내부)   | 해당 데이터만 **DLQ**로 가고, 파이프라인은 **계속 실행**됨 🏃‍♂️     |
| **네트워크/서버 오류** (예: S3 연결 끊김)   | `reader.fetch_data()` | `async_run_pipeline`의 `except`에 걸려 **전체 중단**됨 🛑 |

### 비동기 심화 개념
- asyncio.gather(병렬 실행):
    + 여러 개의 작업을 한꺼번에 실행하고 결과를 모으는 방법
    + 데이터가 100개라면 100개의 요청을 동시에 할 수 있다.
- asyncio.Semaphore(동시성 제한):
    + 동시에 요청할 시 서버가 과부화될 수 있기에 최대 요청 개수를 제한하는 방법
    + 데이터 100개 중 최대 5개씩 요청하도록 제한 

In [25]:
# 세마포어 예시
import asyncio

# 한 번에 5개까지만 허용하는 세마포어 생성
sem = asyncio.Semaphore(5)

async def limited_process_item(item, model_class):
    # 세마포어 안에서만 작업이 실행되도록 제한합니다.
    async with sem:
        # 이 안에서는 최대 5개의 요청이 발생 중입니다.
        return await process_item(item, model_class)

In [26]:
# 완성형 비동기 파이프라인 구조 예시
import asyncio
from typing import List

class AsyncAPIReader:
    async def fetch_data(self) -> List[dict]:
        # 실제는 API 호출을 하지만 예시이므로 비동기 시뮬레이션을 진행합니다.
        printf("API 서버에서 데이터를 가져오는 중...")
        await asyncio.sleep(1) # 네트워크 대기 시간 1초
        return [
            {"id": 1, "name": "키보드", "price": 35000},
            {"id": 2, "name": "마우스", "price": -500}, # Error Data !
            {"id": 3, "name": "모니터", "price": 250000}
        ]

In [27]:
sem = asyncio.Semaphore(3)

async def process_item_with_sem(raw_dict: dict, model_class: Type[T]):

    # 세마포어를 통해 '동시 실행' 개수를 제한합니다.
    async with sem:
        try:
            # 실제 환경이라면 여기서 'await'가 필요한 추가 작업(예시: DB 저장)이 올 수 있습니다.
            instance = model_class(**raw_dict)
            return instance
        except ValidationError:
            return "ERROR_DATA"

In [28]:
# 비동기 파이프라인 구현 예시
import asyncio
from typing import List, Type, TypeVar, Tuple
from pydantic import BaseModel, ValidationError

T = TypeVar("T", bound=BaseModel)

class AsyncPipeline:
    def __init__(self, model_class: Type[T], concurrency_limit: int = 5):
        self.model_class = model_class
        # 동시성 조절
        self.sem = asyncio.Semaphore(concurrency_limit)

    async def _process_item(self, raw_data: dict) -> Tuple[bool, T | dict]:
        """ 개별 데이터 검증 및 변환 (B 단계: DLQ) """
        async with self.sem:
            try:
                # Pydantic 모델로 변환 (타입 강압 및 필드 검증)
                instance = self.model_class(**raw_data)
                return True, instance
            except ValidationError:
                return False, raw_data

        async def run(self, reader) -> Tuple[List[T], List[dict]]:
            """ 전체 파이프라인 실행 (A 단계: 전체 흐름 제어) """
            try:
                # 1. 데이터 추출(Extract)
                raw_items = await reader.fetch_data()

                # 2. 비동기 변환(Transform)
                tasks = [self._process_item(item) for item in raw_items]
                results = await asyncio.gather(*tasks)

                # 3. 결과 분류 (성공 vs DLQ)
                success = [res for ok, res in results if ok]
                errors = [res for ok, res in results if not ok]

                return success, errors

            except Exception as e:
                # 시스템 레벨 에러 처리 (네트워크 단절 등)
                print(f"파이프라인 치명적 오류: {e}")
                raise

In [29]:
import asyncio
from typing import List, Type, TypeVar, Tuple
from pydantic import BaseModel, Field, ValidationError

T = TypeVar("T", bound=BaseModel)

# 1. 데이터 모델 정의 (검증 규칙)
class Product(BaseModel):
    id: int
    name: str = Field(min_length=2)
    price: int = Field(gt=0)

# 2. 비동기 리더 시뮬레이션
class AsyncAPIReader:
    async def fetch_data(self) -> List[dict]:
        await asyncio.sleep(1) # 네트워크 대기 시뮬레이션
        return [
            {"id": 1, "name": "키보드", "price": 35000},
            {"id": 2, "name": "M", "price": 500},      # name 너무 짧음 (에러)
            {"id": 3, "name": "마우스", "price": -500},  # price 음수 (에러)
            {"id": 4, "name": "모니터", "price": 250000}
        ]

# 3. 비동기 파이프라인 클래스
class AsyncPipeline:
    def __init__(self, model_class: Type[T], concurrency_limit: int = 2):
        self.model_class = model_class
        self.sem = asyncio.Semaphore(concurrency_limit)

    async def _process_item(self, raw_data: dict) -> Tuple[bool, T | dict]:
        async with self.sem:
            try:
                instance = self.model_class(**raw_data)
                return True, instance
            except ValidationError:
                return False, raw_data

    async def run(self, reader) -> Tuple[List[T], List[dict]]:
        raw_items = await reader.fetch_data()
        tasks = [self._process_item(item) for item in raw_items]
        results = await asyncio.gather(*tasks)
        
        success = [res for ok, res in results if ok]
        errors = [res for ok, res in results if not ok]
        return success, errors

# 4. 실행부
async def main():
    pipeline = AsyncPipeline(Product)
    reader = AsyncAPIReader()
    
    success, dlq = await pipeline.run(reader)
    
    print(f"✅ 성공 데이터 ({len(success)}건):", success)
    print(f"❌ DLQ 데이터 ({len(dlq)}건):", dlq)

# 주피터 노트북에서는 아래와 같이 실행합니다.
await main()

✅ 성공 데이터 (2건): [Product(id=1, name='키보드', price=35000), Product(id=4, name='모니터', price=250000)]
❌ DLQ 데이터 (2건): [{'id': 2, 'name': 'M', 'price': 500}, {'id': 3, 'name': '마우스', 'price': -500}]


## 11. 데이터 통계 포털 api 기반 파이프라인 설계

In [30]:
# api 키를 숨기기 위해 .env 파일을 만들어 별도로 관리
# python-dotenv 라이브러리를 활용해 키 가져오기

import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("Solar_API_KEY")

print(f"로드된 키 확인: {api_key[:5]}...") # 보안을 위해 앞 5자리까지만 확인

로드된 키 확인: 5ndtX...


In [31]:
# Decoding Key: 원본 열쇠(비압축)
# Encoding Key: 포장 열쇠(압축)
# Request 라이브러리는 압축을 해서 데이터를 받으므로 비압축 상태로 받는게 좋다.

# API 호출에 필요한 파라미터 딕셔너리
params = {
    "serviceKey": api_key, #
    "page": 1,
    "size": 1,
    "startD": "20250101",
    "endD": "20250102"
}

In [32]:
import httpx

async def fetch_solar_data():
    url = "https://apis.data.go.kr/B551893/solar-power-by-hour/list"
    
    # 비동기 클라이언트를 엽니다.
    async with httpx.AsyncClient() as client:
        # API에 데이터를 요청합니다.
        response = await client.get(url, params=params)
        
        # 결과가 JSON 형식인지 확인하고 출력해봅니다.
        # print("--- 서버에서 온 응답 원문 ---")
        # print(response.text)
        # print("---------------------------")
        return response.json()

# 실행
result = await fetch_solar_data()
print(result)

{'reponse': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'content': [{'dgenYmd': '2025-01-01', 'ippt': '84S1', 'hogi': '1', 'ipptNam': '영동태양광', 'qhorGen01': 0, 'qhorGen02': 0, 'qhorGen03': 0, 'qhorGen04': 0, 'qhorGen05': 0, 'qhorGen06': 0, 'qhorGen07': 0, 'qhorGen08': 0, 'qhorGen09': 0, 'qhorGen10': 17.856, 'qhorGen11': 20.544, 'qhorGen12': 20.16, 'qhorGen13': 20.448, 'qhorGen14': 21.696, 'qhorGen15': 396.768, 'qhorGen16': 490.08, 'qhorGen17': 328.128, 'qhorGen18': 549.6, 'qhorGen19': 689.28, 'qhorGen20': 688.32, 'qhorGen21': 285.408, 'qhorGen22': 0, 'qhorGen23': 0, 'qhorGen24': 0, 'qsum': 3528.288, 'qavg': 147, 'qvodMaxS': 689.28, 'qvodMinS': 17.856, 'qvodMax': 689.28, 'qvodMin': 0, 'eaidml': None, 'eaiflag': None, 'eaimsg': None}], 'totalPages': 46, 'last': False, 'totalElements': 46, 'numberOfElements': 1, 'first': True, 'sort': None, 'size': 1, 'number': 0}}}


In [33]:
# 데이터 저장하기

import csv

# 1. 데이터 리스트 추출
items = result['reponse']['body']['content']

# 2. CSV 형태 작성
if items:
    # CSV 헤더(컬럼명) 가져오기
    headers = items[0].keys()

    # 파일을 쓰기 모드로 열기 (newline='' = 윈도우 줄 바꿈 문제 방지)
    with open('solar_power_data_std.csv', 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=headers)

        # 헤더 쓰기
        writer.writeheader()

        # 데이터 행 쓸기
        writer.writerows(items)

    print("CSV 저장 완료: solar_power_data_std.csv")
else:
    print("저장할 데이터가 없습니다.")

CSV 저장 완료: solar_power_data_std.csv
